In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.saving import load_model

In [ ]:
ds_test = pd.read_csv('/kaggle/input/emnist-preprocess/test_images.npy', header=None)
ds_train = pd.read_csv('/kaggle/input/emnist-preprocess/test_labels.npy', header=None)

In [ ]:
X_train, y_train = ds_train.iloc[:, 1:].values, ds_train.iloc[:, 0].values
X_test, y_test = ds_test.iloc[:, 1:].values, ds_test.iloc[:, 0].values

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

X_train_cnn = X_train.reshape(-1, 28, 28, 1)
X_test_cnn = X_test.reshape(-1, 28, 28, 1)

y_train_cnn = to_categorical(y_train)
y_test_cnn = to_categorical(y_test)

In [ ]:
import joblib
cnn_model = load_model("/kaggle/input/emnist-train/cnn_model.h5")
svm_model = joblib.load("/kaggle/input/emnist-train/svm_model.pkl")
knn_model = joblib.load("/kaggle/input/emnist-train/knn_model.pkl")

In [ ]:
y_pred_svm = svm_model.predict(X_test)
y_pred_knn = knn_model.predict(X_test)
y_pred_cnn = cnn_model.predict(X_test_cnn)
y_pred_cnn = y_pred_cnn.argmax(axis=1)

In [ ]:
svm_cm = confusion_matrix(y_test, y_pred_svm)
knn_cm = confusion_matrix(y_test, y_pred_knn)
cnn_cm = confusion_matrix(y_test, y_pred_cnn)

In [ ]:
def plot_confusion_matrix(cm, model_name, color):
    plt.figure(figsize=(24, 20))
    
    sns.heatmap(cm, annot=True, fmt='d', cmap=color, xticklabels=True, yticklabels=True, cbar=True)
    plt.title(f'Confusion Matrix: {model_name}')
    plt.xlabel('Predicted Label', labelpad=25)
    plt.ylabel('True Label', labelpad=25)
    
    plt.xticks(rotation=45, ha='right', fontsize=12)
    plt.yticks(rotation=0, fontsize=12)
    
    plt.show()

In [ ]:
plot_confusion_matrix(svm_cm, "SVM Model", color="Blues")
plot_confusion_matrix(knn_cm, "KNN Model", color="Greens")
plot_confusion_matrix(cnn_cm, "CNN Model", color="Reds")

In [ ]:
def show_values(pc, fmt="%.2f", **kw):
    from itertools import zip_longest
    pc.update_scalarmappable()
    ax = pc.axes
    
    for p, color, value in zip_longest(pc.get_paths(), pc.get_facecolors(), pc.get_array(), fillvalue=None):
        x, y = p.vertices[:-2, :].mean(0)
        if np.all(color[:3] > 0.5):
            color = (0.0, 0.0, 0.0)
        else:
            color = (1.0, 1.0, 1.0)
        ax.text(x, y, fmt % value, ha="center", va="center", color=color, **kw)


def cm2inch(*tupl):
    inch = 2.54
    if type(tupl[0]) == tuple:
        return tuple(i/inch for i in tupl[0])
    else:
        return tuple(i/inch for i in tupl)


def heatmap(AUC, title, xlabel, ylabel, xticklabels, yticklabels, figure_width=40, figure_height=20, correct_orientation=False, cmap='RdBu'):
    
    fig, ax = plt.subplots()    
    c = ax.pcolor(AUC, edgecolors='k', linestyle= 'dashed', linewidths=0.2, cmap=cmap)

    ax.set_yticks(np.arange(AUC.shape[0]) + 0.5, minor=False)
    ax.set_xticks(np.arange(AUC.shape[1]) + 0.5, minor=False)

    ax.set_xticklabels(xticklabels, minor=False)
    ax.set_yticklabels(yticklabels, minor=False)

    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)      

    plt.xlim((0, AUC.shape[1]))

    ax = plt.gca()    
    for t in ax.xaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False
    for t in ax.yaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False

    plt.colorbar(c)

    show_values(c)

    if correct_orientation:
        ax.invert_yaxis()
        ax.xaxis.tick_top()       

    fig = plt.gcf()
    fig.set_size_inches(cm2inch(figure_width, figure_height))


def plot_classification_report(classification_report, title='Classification report ', cmap='RdBu'):
    lines = classification_report.split('\n')

    classes = []
    plotMat = []
    support = []
    class_names = []

    for line in lines[2 : (len(lines) - 3)]:
        t = line.strip().split()
        
        if len(t) < 5: 
            continue
        
        classes.append(t[0])
        v = [float(x) for x in t[1:4]]
        support.append(int(t[-1]))
        class_names.append(t[0])
        plotMat.append(v)

    xlabel = 'Metrics'
    ylabel = 'Classes'
    xticklabels = ['Precision', 'Recall', 'F1-score']
    yticklabels = ['{0} ({1})'.format(class_names[idx], sup) for idx, sup in enumerate(support)]
    
    figure_height = max(20, len(class_names) + 7)
    figure_width = 25  

    correct_orientation = False

    heatmap(np.array(plotMat), title, xlabel, ylabel, xticklabels, yticklabels, figure_width, figure_height, correct_orientation, cmap=cmap)

In [ ]:
svm_accuracy = accuracy_score(y_test, y_pred_svm)
knn_accuracy = accuracy_score(y_test, y_pred_knn)
cnn_accuracy = accuracy_score(y_test, y_pred_cnn)
print(f"SVM Accuracy: {svm_accuracy}")
print(f"KNN Accuracy: {knn_accuracy}")
print(f"CNN Accuracy: {cnn_accuracy}")

In [ ]:
svm_c_r = classification_report(y_test, y_pred_svm, output_dict=False)
knn_c_r = classification_report(y_test, y_pred_knn, output_dict=False)
cnn_c_r = classification_report(y_test, y_pred_cnn, output_dict=False)

In [ ]:
plot_classification_report(svm_c_r, title="Classification Report for SVM", cmap='Blues')
plot_classification_report(knn_c_r, title="Classification Report for KNN", cmap='Greens')
plot_classification_report(cnn_c_r, title="Classification Report for CNN", cmap='Reds')

In [ ]:
models = ['SVM', 'KNN', 'CNN']
accuracies = [svm_accuracy, knn_accuracy, cnn_accuracy]

plt.figure(figsize=(8, 5))
plt.bar(models, accuracies, color=['blue', 'green', 'red'])
plt.title('Comparison of Model Accuracies')
plt.ylabel('Accuracy')
plt.xlabel('Model')
plt.show()